<a href="https://colab.research.google.com/github/jeremiahoclark/python-coding-patterns/blob/main/03_functional_programming_patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. Functional Programming Patterns

Functional programming (FP) emphasizes immutable data and pure functions (no side effects), treating functions as first-class citizens (pass them around like data). Python is multi-paradigm: not a pure functional language, but it supports many FP patterns. Using these can lead to concise and expressive code, especially for data transformations or when concurrency needs immutability.

In this section, we explore patterns such as higher-order functions, closures, immutability, and functional pipelines in Python.

## 3.1 Higher-Order Functions and Lambdas

**Pattern Profile:**

- **Name:** Higher-Order Functions (HOF) & Lambda Use
- **Category:** Functional Pattern
- **Difficulty:** Beginner
- **Python Version:** All
- **Dependencies:** None (built-ins like `map`, `filter` optional)

**Problem Statement:** A higher-order function is one that either takes functions as arguments or returns a function. This pattern enables abstraction over behavior. In Python, functions are first-class, meaning you can pass them around like any object. Common built-in HOFs include `map`, `filter`, `sorted(key=...)`, etc. Lambdas (anonymous functions) often come into play to define simple throwaway functions inline.

**Solution Approach:** Embrace passing functions to functions. Instead of, say, writing separate loops for different behaviors, write a generic function that can accept different function arguments to customize behavior. Use lambdas for small, one-off function definitions to keep code succinct.

In [ ]:
# Higher-order function example: a generic apply_twice
def apply_twice(func, value):
    """Applies a function to a value twice in a row."""
    return func(func(value))

# Using apply_twice with a named function
def add_five(x):
    return x + 5

result1 = apply_twice(add_five, 10)  # adds 5 twice -> 20
print("apply_twice add_five:", result1)

# Using apply_twice with a lambda function
result2 = apply_twice(lambda x: x * 2, 3)  # double twice -> 12
print("apply_twice lambda x*2:", result2)

# Using built-in HOF: sorted with a key function (another form of strategy)
names = ["alice", "Bob", "dave", "Carol"]
sorted_case_insensitive = sorted(names, key=lambda s: s.lower())
print("Sorted case-insensitive:", sorted_case_insensitive)

# Using map and filter
nums = [1, 2, 3, 4, 5]
squares = list(map(lambda x: x*x, nums))
evens = list(filter(lambda x: x % 2 == 0, nums))
print("Squares via map:", squares)
print("Evens via filter:", evens)

**Pros and Cons:**

- *Pros:* HOFs allow very abstract and reusable code. Combined with lambdas, they can eliminate boilerplate for simple operations (no need to define a one-use function with `def` if a lambda will do). It encourages thinking in terms of *what* operation to perform, not *how* to loop.
- *Cons:* Overuse of lambdas or point-free style can make code less readable to those unfamiliar with FP idioms. Debugging is slightly harder when logic is in anonymous functions (no name to reference, harder to print or inspect). Also, Python's lambda is limited to an expression (no statements), so sometimes you need a regular function anyway.

**Practice Exercises:**

In [ ]:
# Exercise 1: Write a compose function
def compose(f, g):
    """Returns a function h such that h(x) = f(g(x))"""
    return lambda x: f(g(x))

# Test the compose function
add_one = lambda x: x + 1
double = lambda x: x * 2

# Create a composed function: add_one(double(x))
add_one_then_double = compose(add_one, double)
print("compose(add_one, double)(5):", add_one_then_double(5))  # (5*2)+1 = 11

# Exercise 2: Function composition pipeline
def apply_functions(value, *functions):
    """Apply a series of functions to a value in sequence"""
    result = value
    for func in functions:
        result = func(result)
    return result

# Test the pipeline
result = apply_functions(5, double, add_one, lambda x: x ** 2)
print("Pipeline result:", result)  # ((5*2)+1)^2 = 11^2 = 121

## 3.2 Immutability and Pure Functions

**Pattern Profile:**

- **Name:** Immutability & Pure Function Pattern
- **Category:** Functional Paradigm
- **Difficulty:** Intermediate
- **Python Version:** All (dataclasses from 3.7 for convenience)
- **Dependencies:** `dataclasses` or `typing.NamedTuple` for convenience (optional)

**Problem Statement:** Functional programming tends to avoid mutable state and side effects, which leads to *pure functions* (output depends only on input, no external state changes). In Python, many objects are mutable (lists, dicts, etc.), but we can choose patterns of usage that treat data as immutable, especially when it leads to clearer or safer code (e.g., no accidental modifications).

**Solution Approach:**

- Use tuples or namedtuples/dataclasses with `frozen=True` to represent data that shouldn't change.
- Write functions that return new objects instead of modifying in-place.
- Avoid global variables or reliance on external state within functions.
- Use recursion or iterative constructs that produce new values rather than updating existing ones.

In [ ]:
from dataclasses import dataclass
from typing import NamedTuple

# Using dataclass as an immutable (frozen) structure
@dataclass(frozen=True)
class Point:
    x: float
    y: float

def move_point(p: Point, dx: float, dy: float) -> Point:
    """Pure function: returns a new Point, doesn't modify input"""
    return Point(p.x + dx, p.y + dy)

p1 = Point(0, 0)
p2 = move_point(p1, 5, 3)
print("Original point:", p1, "| Moved point:", p2)

# Alternative using NamedTuple (inherently immutable)
class Vector(NamedTuple):
    x: float
    y: float

    def add(self, other: 'Vector') -> 'Vector':
        """Pure method: returns new Vector"""
        return Vector(self.x + other.x, self.y + other.y)

v1 = Vector(1, 2)
v2 = Vector(3, 4)
v3 = v1.add(v2)
print(f"Vector addition: {v1} + {v2} = {v3}")

In [ ]:
# Demonstrating tuple vs list immutability
def append_immutable(seq, item):
    """Returns a new sequence with item appended (pure function)"""
    if isinstance(seq, tuple):
        return (*seq, item)  # concatenating tuples creates a new tuple
    elif isinstance(seq, list):
        return seq + [item]  # returns new list (note: + on list creates new list)
    else:
        raise TypeError("Unsupported type")

orig_list = [1, 2, 3]
new_list = append_immutable(orig_list, 4)
print("Original list:", orig_list, "| New list:", new_list)

orig_tuple = (1, 2, 3)
new_tuple = append_immutable(orig_tuple, 4)
print("Original tuple:", orig_tuple, "| New tuple:", new_tuple)

# Immutable dictionary operations
def add_to_dict(d: dict, key, value) -> dict:
    """Returns new dictionary with key-value pair added"""
    return {**d, key: value}  # Creates new dict

orig_dict = {'a': 1, 'b': 2}
new_dict = add_to_dict(orig_dict, 'c', 3)
print("Original dict:", orig_dict, "| New dict:", new_dict)

**Practice Exercises:**

In [ ]:
# Exercise: Immutable Stack implementation
class ImmutableStack:
    def __init__(self, items=()):
        self._items = tuple(items)

    def push(self, item):
        """Returns new stack with item pushed"""
        return ImmutableStack((*self._items, item))

    def pop(self):
        """Returns (new_stack, popped_item) or raises if empty"""
        if not self._items:
            raise IndexError("Stack is empty")
        return ImmutableStack(self._items[:-1]), self._items[-1]

    def peek(self):
        """Returns top item without modifying stack"""
        if not self._items:
            raise IndexError("Stack is empty")
        return self._items[-1]

    def is_empty(self):
        return len(self._items) == 0

    def __repr__(self):
        return f"ImmutableStack({list(self._items)})"

# Test the immutable stack
stack1 = ImmutableStack()
stack2 = stack1.push(1)
stack3 = stack2.push(2).push(3)

print("Original stack:", stack1)
print("After push(1):", stack2)
print("After push(2).push(3):", stack3)

stack4, popped = stack3.pop()
print("After pop():", stack4, "| Popped:", popped)
print("Original stack3 unchanged:", stack3)

## 3.3 Functional Pipelines and Composition

**Pattern Profile:**

- **Name:** Functional Pipeline (Chaining transformations)
- **Category:** Functional Pattern
- **Difficulty:** Intermediate
- **Python Version:** All
- **Dependencies:** itertools for some examples (optional)

**Problem Statement:** A common pattern is to apply a series of transformations to data. In functional style, this often appears as a pipeline of functions where the output of one is the input to the next. Python doesn't have a built-in operator for function composition or piping, but one can achieve readable pipelines using nested function calls, generator pipelines, or by utilizing libraries.

**Solution Approach:**

- **Nested calls / composition:** `result = f3(f2(f1(data)))`
- **Using generator pipeline:** break steps into generator functions that yield intermediate results
- **Method chaining with custom classes:** implement classes whose methods return self or new instance
- **Third-party or functools:** e.g., `functools.reduce` could combine functions

In [ ]:
import math
from functools import reduce

# Define some simple transformations
def double(x): return x*2
def square(x): return x*x
def minus_one(x): return x-1

# Pipeline via nested function calls:
num = 3
result = minus_one(square(double(num)))  # double -> square -> minus_one
print("Pipeline nested result:", result)

# Function composition utility (returns a new function that is comp of f and g)
def compose(f, g):
    return lambda *args, **kwargs: f(g(*args, **kwargs))

# Compose multiple functions (right to left)
pipeline_func = reduce(lambda f, g: compose(f, g), [minus_one, square, double])
print("Pipeline via compose:", pipeline_func(3))

# Alternative: left-to-right composition
def pipe(*functions):
    """Creates a pipeline function that applies functions left to right"""
    return lambda x: reduce(lambda acc, f: f(acc), functions, x)

# Create pipeline: double -> square -> minus_one
pipeline = pipe(double, square, minus_one)
print("Left-to-right pipeline:", pipeline(3))

In [ ]:
# Generator pipeline example: generate some numbers, then apply pipeline lazily
def nums(start, end):
    for i in range(start, end):
        yield i

def pipeline_gen(iterable):
    for x in iterable:
        y = double(x)
        y = square(y)
        y = minus_one(y)
        yield y

print("Pipeline via generator:", list(pipeline_gen(nums(1, 5))))

# More functional generator pipeline using map
def functional_pipeline(iterable):
    """Chain transformations using map for a functional approach"""
    step1 = map(double, iterable)
    step2 = map(square, step1)
    step3 = map(minus_one, step2)
    return step3

print("Functional generator pipeline:", list(functional_pipeline(range(1, 5))))

In [ ]:
# Method chaining example (like pandas)
class StringProcessor:
    def __init__(self, text: str):
        self.text = text

    def to_lower(self):
        self.text = self.text.lower()
        return self  # return self to allow chaining

    def remove_punct(self):
        self.text = "".join(ch for ch in self.text if ch.isalnum() or ch.isspace())
        return self

    def trim(self):
        self.text = self.text.strip()
        return self

    def replace_spaces(self, replacement="_"):
        self.text = self.text.replace(" ", replacement)
        return self

proc = StringProcessor("  Hello, World!! ")
result_text = proc.to_lower().remove_punct().trim().replace_spaces().text
print("Chained methods result:", result_text)

# Immutable version (returns new instances)
class ImmutableStringProcessor:
    def __init__(self, text: str):
        self.text = text

    def to_lower(self):
        return ImmutableStringProcessor(self.text.lower())

    def remove_punct(self):
        clean_text = "".join(ch for ch in self.text if ch.isalnum() or ch.isspace())
        return ImmutableStringProcessor(clean_text)

    def trim(self):
        return ImmutableStringProcessor(self.text.strip())

    def __repr__(self):
        return f"ImmutableStringProcessor('{self.text}')"

original = ImmutableStringProcessor("  Hello, World!! ")
processed = original.to_lower().remove_punct().trim()
print("Original:", original)
print("Processed:", processed)

**Practice Exercises:**

In [ ]:
# Exercise: Data processing pipeline
from itertools import accumulate

# Create a pipeline for processing numerical data
def process_numbers(numbers):
    """Pipeline: filter evens -> square -> running sum"""
    evens = filter(lambda x: x % 2 == 0, numbers)
    squared = map(lambda x: x ** 2, evens)
    running_sums = accumulate(squared)
    return list(running_sums)

data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = process_numbers(data)
print("Processed numbers:", result)

# Exercise: Create a generic pipeline utility
class Pipeline:
    def __init__(self, data):
        self.data = data

    def filter(self, predicate):
        return Pipeline(filter(predicate, self.data))

    def map(self, func):
        return Pipeline(map(func, self.data))

    def reduce(self, func, initial=None):
        if initial is None:
            return reduce(func, self.data)
        return reduce(func, self.data, initial)

    def collect(self):
        return list(self.data)

# Use the pipeline
result = (Pipeline(range(1, 11))
         .filter(lambda x: x % 2 == 0)
         .map(lambda x: x ** 2)
         .collect())

print("Pipeline result:", result)

# Sum using reduce
total = (Pipeline(range(1, 11))
        .filter(lambda x: x % 2 == 0)
        .map(lambda x: x ** 2)
        .reduce(lambda a, b: a + b, 0))

print("Pipeline sum:", total)

## Summary

Functional programming patterns in Python provide powerful tools for writing clean, maintainable code:

1. **Higher-Order Functions and Lambdas**: Enable abstraction over behavior and concise function definitions
2. **Immutability and Pure Functions**: Reduce bugs and make code easier to reason about
3. **Functional Pipelines**: Create clear data transformation workflows

While Python isn't a purely functional language, incorporating these patterns can lead to more robust and expressive code, especially in data processing, concurrent programming, and mathematical computations.

**Key Takeaways:**
- Use functions as first-class objects to create flexible, reusable code
- Prefer immutable data structures when possible to avoid side effects
- Chain operations into pipelines for clear data transformation workflows
- Balance functional patterns with Python's pragmatic approach for optimal readability